In [ ]:
# April 2016
# Debbie Hofman
# Load data from Twitter API into a Cypher graph similar to
# https://gist.github.com/nicolewhite/fb41e76844ce10183849 oscon_twitter_graph.py

In [ ]:
### Data Loader uses Cypher to do ELT process of transforming the raw JSON input into a neo4j Graph
### Written for neo4j v. 2.3.3 using py2neo library

In [3]:
#!wc -l "./archive/data/election172"
!cat "./archive/data/election172" | head

cat: stdout: Broken pipe


In [5]:
%%writefile cypher_data_loader.py

import sys, getopt
import py2neo as neo4j
import json
import datetime
import math
import time

epoch = datetime.datetime.utcfromtimestamp(0)

def readargs(argv):
    
    inputfile=''
    password=''
    remoteserver=''
    maxsize=600000
    batchsize=1000
    partition_key='election'
    constraints=False
    partitionsize=0
    startval=0
    
    try:
        opts, args = getopt.getopt(argv,"hi:p:r:m:b:k:z:s:c")
    except getopt.GetoptError:
        print 'neo2.py -i <inputfile> -p <password for local server> -r <remote server url> \
        -m <maxnumberrows> -b <batchsize> -k <partition_key> -s <startval>'
        sys.exit(2)
    for opt, arg in opts:
        if opt == '-h':
            print 'neo2.py -i <inputfile> -p <password for local server> -r <remote server url> \
            -m <maxnumberrows> -b <batchsize> -k <partition_key> -s <startval>'
            sys.exit()
        elif opt == "-i":
            inputfile = arg
        elif opt == "-p":
            password = arg
        elif opt == "-r":
            remoteserver = arg
        elif opt == "-m":
            maxsize = int(arg)
        elif opt == "-b":
            batchsize = int(arg)
        elif opt == "-k":
            partition_key = arg
        elif opt == "-c":
            constraints = True
        elif opt == "-z":
            partitionsize = arg
        elif opt == "-s":
            startval=int(arg)
            
    return inputfile, password, remoteserver, maxsize, batchsize, partition_key, constraints, partitionsize, startval

def authenticate(password, remoteserver):
    mode='none'
    try:
        if ((password=='') and (remoteserver=='')):
            mode='no auth'
            graph = neo4j.Graph()
            return graph

        elif (remoteserver==''):
            mode='password local'
            pyn.authenticate("localhost:7474","neo4j",password)
            graph = neo4j.Graph()
            return graph

        elif (remoteserver!=''):
            mode='remote server'
            remote_graph = neo4j.Graph(remoteserver)
            return remote_graph

    except:
        print 'neo2.py problem with authentication for mode ' + mode
        sys.exit(2)
    
    return
   
def unix_time_millis(dt):
    return int( (dt - epoch).total_seconds() * 1000.0)

def addtweet(data,databatch):
    
    pretweet=''
    
    if (('retweeted' in data) and ('retweeted_status' in data)):
        pretweet = data['retweeted_status'];

    elif (('quoted_status' in data)):
        pretweet = data['quoted_status'];
        
    if (pretweet != ''):
        databatch['tweets'].append(pretweet) 
    
    databatch['tweets'].append(data)

#create roughly hourly partition
def getPartition(timestamp_ms):
    return int(timestamp_ms / (3600 * 1000)) # divide milliseconds to get hours

def dropConstraints(graph):
    
    constraints = [
        "DROP CONSTRAINT ON (t:Tweet) ASSERT t.id_str IS UNIQUE;",
        "DROP CONSTRAINT ON (u:User) ASSERT u.screen_name IS UNIQUE;",
        "DROP CONSTRAINT ON (h:Hashtag) ASSERT h.name IS UNIQUE;",
        "DROP CONSTRAINT ON (l:Link) ASSERT l.url IS UNIQUE;",
        "DROP CONSTRAINT ON (s:Source) ASSERT s.source IS UNIQUE;",
        "DROP CONSTRAINT ON (p:Place) ASSERT p.id IS UNIQUE;",
        "DROP CONSTRAINT ON (t:Tweet) ASSERT t.id IS UNIQUE;",
        "DROP CONSTRAINT ON (t:Tweet) ASSERT t.partition_key IS UNIQUE;",
        "DROP CONSTRAINT ON (u:User) ASSERT u.partition_key IS UNIQUE;",
        "DROP CONSTRAINT ON (h:Hashtag) ASSERT h.partition_key IS UNIQUE;",
        "DROP CONSTRAINT ON (l:Link) ASSERT l.partition_key IS UNIQUE;",
        "DROP CONSTRAINT ON (s:Source) ASSERT s.partition_key IS UNIQUE;",
        "DROP CONSTRAINT ON (p:Place) ASSERT p.partition_key IS UNIQUE;"
    ]
    for constraint in constraints:
        try:
            graph.cypher.execute(constraint)
            print constraint
        except Exception as e:
            print 'Could not drop the following constraint -- ', constraint

def setConstraints(graph, partition_key):
    constraints = [
        """CREATE CONSTRAINT ON (t:Tweet_{0}) ASSERT t.id_str IS UNIQUE;""".format(partition_key),
        """CREATE CONSTRAINT ON (u:User_{0}) ASSERT u.screen_name IS UNIQUE;""".format(partition_key),
        """CREATE CONSTRAINT ON (h:Hashtag_{0}) ASSERT h.name IS UNIQUE;""".format(partition_key),
        """CREATE CONSTRAINT ON (l:Link_{0}) ASSERT l.url IS UNIQUE;""".format(partition_key),
        """CREATE CONSTRAINT ON (s:Source_{0}) ASSERT s.source IS UNIQUE;""".format(partition_key),
        """CREATE CONSTRAINT ON (p:Place_{0}) ASSERT p.id IS UNIQUE;""".format(partition_key),
        """CREATE INDEX ON :Tweet_{0}(timestamp_int);""".format(partition_key)
    ]
    for constraint in constraints:
        try:
            graph.cypher.execute(constraint)
            print constraint
        except Exception as e:
            print 'Could not create the following constraint -- ', constraint


def tweetQuery(partition_key):
    
    tweetstr = """MERGE (tweet:Tweet_{0}:Tweet {{id_str:t.id_str}})
        ON CREATE SET tweet.text = t.text,
            tweet.created_at = t.created_at,
            tweet.favorite_count = t.favorite_count,
            tweet.timestamp_int = toInt(t.timestamp_ms)
        ON MATCH SET tweet += {{
            favorite_count:t.favorite_count
        }}
        FOREACH (coord IN [c IN [t.coordinates] WHERE c IS NOT NULL] |
            SET tweet.coordinates = coord.coordinates
        )""".format(partition_key)
#     print 'tweet', tweetstr
    return tweetstr

def userQuery(partition_key):
    
    userstr = """
        MERGE (user:User_{0}:User {{screen_name:u.screen_name}})
        ON CREATE SET user.name = u.name,
            user.location = u.location,
            user.followers = u.followers_count,
            user.description = u.description,
            user.following = u.friends_count,
            user.time_zone = u.time_zone,
            user.statuses_count = u.statuses_count,
            user.verified = u.verified
        ON MATCH SET user += {{
            location : u.location,
            followers : u.followers_count,
            description : u.description,
            following : u.friends_count,
            time_zone : u.time_zone,
            statuses_count : u.statuses_count,
            verified : u.verified
        }}
        MERGE (user)-[:POSTS]->(tweet)""".format(partition_key)
#     print 'user', userstr
    return userstr

def placeQuery(partition_key):
    
    placestr = """
        FOREACH (pid IN p.id |
            MERGE (place:Place_{0}:Place {{id:p.id}})
            ON CREATE SET place.url=p.url,
                place.id_str=p.id,
                place.place_type=p.place_type,
                place.country=p.country,
                place.name=p.name,
                place.full_name=p.full_name,
                place.country_code=p.country_code,
                place.country=p.country
            MERGE (tweet)-[:LOCATED_IN]->(place)
        )""".format(partition_key)
#     print 'place', placestr
    return placestr

def sourceQuery(partition_key):
    
    sourcestr = """
        MERGE (source:Source_{0}:Source {{source:t.source}})
        MERGE (tweet)-[:USING]->(source)""".format(partition_key)
#     print 'source', sourcestr
    return sourcestr

def entitiesQuery(partition_key):
    
    entitiesstr = """
        FOREACH (h IN e.hashtags |
            MERGE (tag:Hashtag_{0}:Hashtag {{name:LOWER(h.text)}})
            MERGE (tag)-[:TAGS{{indices:h.indices}}]->(tweet)
        )
        FOREACH (u IN e.urls |
            MERGE (url:Link_{0}:Link {{url:u.expanded_url}})
            MERGE (tweet)-[:CONTAINS{{indices:u.indices}}]->(url)
        )
        FOREACH (m IN e.user_mentions |
            MERGE (mentioned:User_{0}:User {{screen_name:m.screen_name}})
            ON CREATE SET mentioned.name = m.name
            MERGE (tweet)-[:MENTIONS{{indices:m.indices}}]->(mentioned)
        )""".format(partition_key)
#     print 'entitities', entitiesstr
    return entitiesstr

def replytoQuery(partition_key):
    
    replytostr = """
        FOREACH (r IN [r IN [t.in_reply_to_status_id_str] WHERE r IS NOT NULL] |
            MERGE (reply_tweet:Tweet_{0}:Tweet {{id_str:r}})
            MERGE (tweet)-[:REPLY_TO]->(reply_tweet)
        )""".format(partition_key)
#     print 'replyto', replytostr
    return replytostr

def retweetQuery(partition_key):
    
    retweetstr = """
    FOREACH (retweet_id IN [x IN [retweet.id_str] WHERE x IS NOT NULL] |
            MERGE (retweet_tweet:Tweet_{0}:Tweet {{id_str:retweet_id}})
            MERGE (tweet)-[:RETWEETS]->(retweet_tweet)
        )""".format(partition_key)
#     print 'retweet', retweetstr
    return retweetstr
        
def runQuery(graph,tweets, partition_key):
    
# Pass dict to Cypher and build query.
    query = """
        WITH {json} as data
        UNWIND data.tweets as t 
        WITH t
        ORDER BY t.id
        WITH t,
             t.entities AS e,
             t.user AS u,
             t.place AS p,
             t.retweeted_status AS retweet
    """\
    + tweetQuery(partition_key)\
    + userQuery(partition_key)\
    + placeQuery(partition_key)\
    + sourceQuery(partition_key)\
    + entitiesQuery(partition_key)\
    + replytoQuery(partition_key)\
    + retweetQuery(partition_key)
    
    #print query
    
    try:
        params = dict(json=tweets)
        graph.cypher.execute(query, params)
        
    except Exception as e:
        print(e)
        
def writebatch(graph, linescount, start, databatch,partition_key,partitionsize):
    try:

        if (partitionsize > 0):
            partition_key = partition_key + '_' + str(linescount/partitionsize)
        
        runQuery(graph,databatch,partition_key)
                            
        databatch['tweets'] = []
                            
        print "{0} lines processed after {1} seconds.".format(linescount,time.time()-start)
   
    except:
        print "error at record {0}".format(linescount)
    
def main(argv):
    
    fname, password, remoteserver, readsize, batchsize, partition_key, constraints, partitionsize, startval = readargs(argv)
    print partition_key

    graph = authenticate(password, remoteserver)
    
    start = time.time()

# Connect to graph and add constraints.

    if (constraints):
        #dropConstraints(graph)
        setConstraints(graph, partition_key)
        
    databatch = {'tweets':[]}

    with open(fname, 'rb') as f:
        i = 0;
        j = 0;
  
        for line in f:  
                
            if ((line is None) or (j >= readsize)): #test end of file, write remaining data
                if (i>0):
                    writebatch (graph, j, start, databatch,partition_key,partitionsize)
                break

            j=j+1
            
            if (j >= startval):
                
                i=i+1

                try:

                    data = json.loads(line, encoding='utf8')

                    if ('limit' not in data.keys()):
                        addtweet(data,databatch)

                    if ((i == batchsize) or (i==readsize)): 
                        writebatch (graph, j, start, databatch,partition_key,partitionsize)
                        i = 0    

                except Exception as e:
                    print 'final error encountered...', e, j, "lines processed"
    
        end = time.time()
        print end - start
        
        
if __name__ == "__main__":
    main(sys.argv[1:])
    

Overwriting cypher_data_loader.py


In [7]:
# !python json_twitter_loader.py -i 'data/election172' -r 'http://localhost:7474/db/data' \
# -m 20000000 -b 1000 -k '172' -c


In [22]:


for i in ['191']:
    
    filename = './archive/data/election{0}'.format(i)
    partitionkey = '{0}'.format(i)
    
    !python cypher_data_loader.py -i $filename \
    -r 'http://localhost:7474/db/data' \
    -s 717000 -m 1000000 -b 1000 -k $partitionkey



191
717999 lines processed after 22.6590359211 seconds.
718999 lines processed after 50.6272208691 seconds.
719999 lines processed after 73.9657549858 seconds.
720999 lines processed after 94.8544158936 seconds.
721999 lines processed after 115.225142956 seconds.
722999 lines processed after 141.238190889 seconds.
723999 lines processed after 165.504703999 seconds.
724999 lines processed after 186.799711943 seconds.
725999 lines processed after 210.359490871 seconds.
726999 lines processed after 231.040259838 seconds.
727999 lines processed after 255.038166046 seconds.
728999 lines processed after 275.277183056 seconds.
729999 lines processed after 294.837776899 seconds.
730999 lines processed after 312.347166061 seconds.
731999 lines processed after 332.313933849 seconds.
732999 lines processed after 351.764287949 seconds.
733999 lines processed after 376.340904951 seconds.
734999 lines processed after 397.749153852 seconds.
735999 lines processed after 420.891018867 seconds.
736999 l